In [4]:

import requests
import time
import scrapy

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from scrapy.http import TextResponse
from bs4 import BeautifulSoup

import pymongo


In [5]:

options = Options()
options.headless = True
driver = webdriver.Chrome('/home/ubuntu/chromedriver', options=options)
driver.get('https://www.youtube.com/results?search_query=%ED%83%80%EB%8B%A4+%EA%B8%88%EC%A7%80%EB%B2%95&sp=CAMSBAgDEAE%253D')

page=driver.page_source
soup = BeautifulSoup(page,'lxml')

all_ = soup.find_all('a','yt-simple-endpoint style-scope ytd-video-renderer')

#제목
titles = [all_[n].text.strip() for n in range(0,len(all_))]
#주소 
links= ["https://www.youtube.com/" + all_[n].get('href') for n in range(0,len(all_))]

titles, links

(['국회 통과된 타다금지법을 즉시 시행하라 20.03.07-3',
  "'타다 베이직' 4월 11일 중단…1만명 넘는 드라이버 어쩌나 / JTBC 뉴스룸",
  '[KBS열린토론] 타다 금지법 이후, 차량 공유서비스는 어디로? / 정준희, 구태언, 권용주, 최성진(200310)',
  "타다, 4월 10일까지만 운행…드라이버 1만여 명 '생계 막막' / JTBC 아침&",
  '타다 베이직, 다음 달 11일부터 잠정 중단 / 연합뉴스TV (YonhapnewsTV)',
  '‘타다 종료’ 누구를 위한 타다 금지법인가ㅣ쏘카, 타다, 모빌리티 [롱앵커]',
  '"변호사가 보는 타다금지법" 타다는 혁신이다? VS 꼼수다?',
  '타다, 파견 직원들 권고사직 요구…감원 착수 / 연합뉴스TV (YonhapnewsTV)',
  '타다 타다금지법 통과 이후... 청와대 호소??? 04.11 서비스 종료???',
  '타다 안녕, 택시 시장 넘보지 마라말이야! - 타다금지법 이 아니고 여객자동차운수사업법 개정안이 국회를 통과했습니다.',
  '타다금지법에 대한 내 생각',
  "고객이 마케팅해주는 기업 최고의 서비스, 저렴한 가격으로 사랑받던 '타다' 국회 본회의 통과... 타다금지법/ 후폭풍...",
  '혁신산업의 싹을 자른 국회ㅣ타다, 이대로 사라지나?ㅣ채이배 민생당 국회의원ㅣ시대전환의 소리',
  '#3 타다 택시 상생은 어려운 것일까, IT혁신을 가로막는 타다금지법 타다 드라이버 인터뷰',
  '타다 금지법과 인터넷전문은행법 개정안, 혁신을 대하는 이중 잣대?ㅣ채이배 민생당 의원',
  "[보이는라디오] '타다금지법' 국회통과, 혁신의 한계인가 상생인가 3/10(화) 뉴스FM, 조현지입니다 YTN RADIO",
  'VOF#4-규제개혁당 "한국은 규제개혁 스타트업"',
  '타다금지법 타다는 불법인가? 혁신인가? 카카오는 이미 알고 있었다.',
  '타다금지법 통과…타다 이슈를 끝맺는 소회 / 코로나 이후 중국에서는 어떤 사업이 뜰 것인가_IT Issue Wee

In [6]:
# 첫 영상의 댓글이 0일 경우 에러가 발생한다.
comment_data = pd.DataFrame({'제목': [],
                             'user_id': [],
                             '댓글': [],
                             '좋아요 수': []
                             })

for i in range(0, 3):
    start_url = links[i]
    #driver = webdriver.Chrome('/home/ubuntu/chromedriver', options=options)
    driver.get(start_url)
    
    body = driver.find_element_by_tag_name("body")

    num_of_pagedowns = 10
    
    while num_of_pagedowns: 
        body.send_keys(Keys.PAGE_DOWN) 
        time.sleep(2)
        num_of_pagedowns -= 1 
    
    try: 
        driver.find_element_by_xpath('//*[@id="sort-menu"]').click() 
        driver.find_element_by_xpath('//*[@id="menu"]/a[2]/paper-item/paper-item-body/div[text()="최근 날짜순"]').click() 
    
    except Exception as e: 
        pass
    #제목
    title = driver.find_element_by_css_selector('#container > h1 > yt-formatted-string').text
    #사용자 id    
    user_ids = driver.find_elements_by_css_selector('#author-text > span')
    # 댓글
    comments = driver.find_elements_by_css_selector('#content-text')
    #좋아요 수
    like_nums = driver.find_elements_by_css_selector('#vote-count-middle')
    
    datas = []

    for j in range(len(comments)):
        title = title
        comment = comments[j].text
        user_id = user_ids[j].text
        like_num = like_nums[j].text
        
        datas.append({
        "제목" : title,
        "user_id" : user_id,
        "댓글" : comment,
        "좋아요 수" : like_num
         })

    result_data=pd.DataFrame(datas)
    result = result_data[["제목", "user_id", "댓글", "좋아요 수"]]
    comment_data = comment_data.append(result, ignore_index=True)

In [7]:
comment_data


,제목,user_id,댓글,좋아요 수
0,국회 통과된 타다금지법을 즉시 시행하라 20.03.07-3,chulsoo kim,"사람들이 모르는 거...자가용 운전자가 자신이 아는 사람 태우고 갈 경우, 사고가...",9
1,국회 통과된 타다금지법을 즉시 시행하라 20.03.07-3,duksoo kim,기사가 운전하는 타다는 택시와 달리 승객의 지위가 서비스를 받는 손님이 아니라 기사...,9
2,국회 통과된 타다금지법을 즉시 시행하라 20.03.07-3,박현숙,최소한의 규칙까지도 무너뜨리려는 행위\n하는짓들이 하나같이 문제투성이네요,10
3,국회 통과된 타다금지법을 즉시 시행하라 20.03.07-3,김태민,타다법 법사위 통과는 불법편법을 4차산업이라는 말을 내세워 합법화처럼 운영하려했던 ...,4
4,국회 통과된 타다금지법을 즉시 시행하라 20.03.07-3,justice lee,이재웅이 친노돌과 짬짜미한 것으로\n의심됨,4
5,국회 통과된 타다금지법을 즉시 시행하라 20.03.07-3,이대구,타다\n잘몰랐는데\n야메\n한방에 알았네요 ~~~꾸벅,8
6,국회 통과된 타다금지법을 즉시 시행하라 20.03.07-3,bum tiger,"""타다"" 잘 몰랐는데, 이쯤되면 \n타다 국정조사해야 되는 것 아닌가? \n뭔가 검...",2
7,국회 통과된 타다금지법을 즉시 시행하라 20.03.07-3,박현숙,타다금지법 통과\n너무 잘됐네요^^,9
8,국회 통과된 타다금지법을 즉시 시행하라 20.03.07-3,야누스,백두산에 가서 문재인하고 백두산수 먹었던 타다 사장님\n결국 물 먹었네요,4
9,국회 통과된 타다금지법을 즉시 시행하라 20.03.07-3,박병춘,함께 힘냅시다 힘,5


#### Mongo DB 넣기

In [8]:
# Mongo - id pw 입력 후 실행 port 27017
import pymongo
client = pymongo.MongoClient('mongodb://id:pw@ip.ip.ip.ip:port/')


In [ ]:
# IP 정보가 나와 실행하지 않음... 실행하면 정보 나옴!
comment_result = client.crawling.youtube
comment_result

In [10]:
#dict 형태 변경
items = comment_data.to_dict("records")

In [11]:
# 데이터 넣기..
ids = comment_result.insert(items)

In [13]:
# 확인
items[0]['제목']

'국회 통과된 타다금지법을 즉시 시행하라 20.03.07-3'

In [14]:
# DataFrame으로 보고 싶을 때
pd.DataFrame(comment_result.find({}))

,_id,user_id,댓글,제목,좋아요 수
0,5e6e44fc899b7f0e56f49472,chulsoo kim,"사람들이 모르는 거...자가용 운전자가 자신이 아는 사람 태우고 갈 경우, 사고가...",국회 통과된 타다금지법을 즉시 시행하라 20.03.07-3,9
1,5e6e44fc899b7f0e56f49473,duksoo kim,기사가 운전하는 타다는 택시와 달리 승객의 지위가 서비스를 받는 손님이 아니라 기사...,국회 통과된 타다금지법을 즉시 시행하라 20.03.07-3,9
2,5e6e44fc899b7f0e56f49474,박현숙,최소한의 규칙까지도 무너뜨리려는 행위\n하는짓들이 하나같이 문제투성이네요,국회 통과된 타다금지법을 즉시 시행하라 20.03.07-3,10
3,5e6e44fc899b7f0e56f49475,김태민,타다법 법사위 통과는 불법편법을 4차산업이라는 말을 내세워 합법화처럼 운영하려했던 ...,국회 통과된 타다금지법을 즉시 시행하라 20.03.07-3,4
4,5e6e44fc899b7f0e56f49476,justice lee,이재웅이 친노돌과 짬짜미한 것으로\n의심됨,국회 통과된 타다금지법을 즉시 시행하라 20.03.07-3,4
5,5e6e44fc899b7f0e56f49477,이대구,타다\n잘몰랐는데\n야메\n한방에 알았네요 ~~~꾸벅,국회 통과된 타다금지법을 즉시 시행하라 20.03.07-3,8
6,5e6e44fc899b7f0e56f49478,bum tiger,"""타다"" 잘 몰랐는데, 이쯤되면 \n타다 국정조사해야 되는 것 아닌가? \n뭔가 검...",국회 통과된 타다금지법을 즉시 시행하라 20.03.07-3,2
7,5e6e44fc899b7f0e56f49479,박현숙,타다금지법 통과\n너무 잘됐네요^^,국회 통과된 타다금지법을 즉시 시행하라 20.03.07-3,9
8,5e6e44fc899b7f0e56f4947a,야누스,백두산에 가서 문재인하고 백두산수 먹었던 타다 사장님\n결국 물 먹었네요,국회 통과된 타다금지법을 즉시 시행하라 20.03.07-3,4
9,5e6e44fc899b7f0e56f4947b,박병춘,함께 힘냅시다 힘,국회 통과된 타다금지법을 즉시 시행하라 20.03.07-3,5


In [39]:
# 드롭하고 싶을 때.
client.crawling.drop_collection("youtube")

In [42]:
client.drop_database("crawling")

In [15]:
#####################mongodb 저장 끝
#####################scrapy 미완성 

#### Scrapy - 미완성

In [34]:
!rm -rf youcraw

In [35]:
!scrapy startproject you_crawal

New Scrapy project 'you_crawal', using template directory '/home/ubuntu/.pyenv/versions/3.6.9/envs/python3/lib/python3.6/site-packages/scrapy/templates/project', created in:
    /home/ubuntu/crawling_project/you_crawal

You can start your first spider with:
    cd you_crawal
    scrapy genspider example example.com


In [1]:
!tree you_crawal

you_crawal
├── scrapy.cfg
├── you_crawal
│   ├── __init__.py
│   ├── __pycache__
│   │   ├── __init__.cpython-36.pyc
│   │   ├── items.cpython-36.pyc
│   │   └── settings.cpython-36.pyc
│   ├── items.py
│   ├── middlewares.py
│   ├── mongodb.py
│   ├── pipelines.py
│   ├── settings.py
│   └── spiders
│       ├── __init__.py
│       ├── __pycache__
│       │   ├── __init__.cpython-36.pyc
│       │   └── spider.cpython-36.pyc
│       └── spider.py
└── you_crwal.csv

4 directories, 15 files


In [36]:
!cat you_crawal/you_crawal/items.py

# -*- coding: utf-8 -*-

# Define here the models for your scraped items
#
# See documentation in:
# https://docs.scrapy.org/en/latest/topics/items.html

import scrapy


class YouCrawalItem(scrapy.Item):
    # define the fields for your item here like:
    # name = scrapy.Field()
    pass


In [71]:
%%writefile you_crawal/you_crawal/items.py
import scrapy


class YouCrawalItem(scrapy.Item):
    links = scrapy.Field()
    title = scrapy.Field()
    comment = scrapy.Field()
    user_id = scrapy.Field()
    like_num = scrapy.Field()

Overwriting you_crawal/you_crawal/items.py


In [82]:
%%writefile you_crawal/you_crawal/spiders/spider.py
import scrapy
import pandas as pd
from you_crawal.items import YouCrawalItem
from selenium import webdriver


class CommentSpider(scrapy.Spider):
    name= "YoutubeComment"
    allow_domain = ["youtube.com"]
    start_urls = ["https://www.youtube.com/results?search_query=타다 금지법&sp=EgIIAw%253D%253D"]
    
    def parse(self, response):
        options = Options()
        options.headless = True
        driver = webdriver.Chrome('/home/ubuntu/chromedriver', options=options)
        driver.get('https://www.youtube.com/results?search_query=타다 금지법&sp=EgIIAw%253D%253D')
        
        page=driver.page_source
        soup = BeautifulSoup(page,'lxml')
        
        all_ = soup.find_all('a','yt-simple-endpoint style-scope ytd-video-renderer')
               
        #주소 
        links= ["https://www.youtube.com/" + all_[n].get('href') for n in range(0,len(all_))]
    
    def parse_links(self, rersponse):
        comment_data = pd.DataFrame({'제목': [],
                             'user_id': [],
                             '댓글': [],
                             '좋아요 수': []
                             })

        for i in range(0, 3):
            start_url = links[i]
            #driver = webdriver.Chrome('/home/ubuntu/chromedriver', options=options)
            driver.get(start_url)
            
            body = driver.find_element_by_tag_name("body")
        
            num_of_pagedowns = 10
            
            while num_of_pagedowns: 
                body.send_keys(Keys.PAGE_DOWN) 
                time.sleep(2)
                num_of_pagedowns -= 1 
            
            try: 
                driver.find_element_by_xpath('//*[@id="sort-menu"]').click() 
                driver.find_element_by_xpath('//*[@id="menu"]/a[2]/paper-item/paper-item-body/div[text()="최근 날짜순"]').click() 
            
            except Exception as e: 
                pass
            
        
    def parse_page_contents(self, response):
        item = YouCrawalItem()
            #제목
        item["title"] = driver.find_element_by_css_selector('#container > h1 > yt-formatted-string').text
        #사용자 id    
        item["user_ids"] = driver.find_elements_by_css_selector('#author-text > span')
        # 댓글
        item["comments"] = driver.find_elements_by_css_selector('#content-text')
        #좋아요 수
        item["like_nums"] = driver.find_elements_by_css_selector('#vote-count-middle')
    
        yield item
    

Overwriting you_crawal/you_crawal/spiders/spider.py


In [83]:
!ls you_crawal/

scrapy.cfg  you_crawal	you_crwal.csv


In [84]:
%%writefile run.sh
#!/bin/bash
cd you_crawal
scrapy crawl YoutubeComment -o you_crwal.csv

Overwriting run.sh


In [80]:
!ls -al run.sh

-rwxrw-r-- 1 ubuntu ubuntu 71 Mar 15 17:16 run.sh


In [52]:
os.chmod("run.sh", 0o764)

In [68]:
!head -n 25 you_crawal/you_crawal/settings.py | tail -n 5

# Obey robots.txt rules
ROBOTSTXT_OBEY = False

# Configure maximum concurrent requests performed by Scrapy (default: 16)
#CONCURRENT_REQUESTS = 32


In [67]:
!sed -i 's/ROBOTSTXT_OBEY = True/ROBOTSTXT_OBEY = False/' you_crawal/you_crawal/settings.py

In [85]:
!./run.sh

2020-03-15 17:17:03 [scrapy.utils.log] INFO: Scrapy 1.8.0 started (bot: you_crawal)
2020-03-15 17:17:03 [scrapy.utils.log] INFO: Versions: lxml 4.5.0.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.5.2, w3lib 1.21.0, Twisted 19.10.0, Python 3.6.9 (default, Feb 11 2020, 12:02:52) - [GCC 7.4.0], pyOpenSSL 19.1.0 (OpenSSL 1.1.1d  10 Sep 2019), cryptography 2.8, Platform Linux-4.15.0-1060-aws-x86_64-with-debian-buster-sid
2020-03-15 17:17:03 [scrapy.crawler] INFO: Overridden settings: {'BOT_NAME': 'you_crawal', 'FEED_FORMAT': 'csv', 'FEED_URI': 'you_crwal.csv', 'NEWSPIDER_MODULE': 'you_crawal.spiders', 'SPIDER_MODULES': ['you_crawal.spiders']}
2020-03-15 17:17:03 [scrapy.extensions.telnet] INFO: Telnet Password: 11bea76570b04f6f
2020-03-15 17:17:03 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats

#### 파이프라인 작성

In [86]:
%%writefile you_crawal/you_crawal/mongodb.py 
import pymongo

# 몽고DB 정보 입력 후 실행 포트 27017
client = pymongo.MongoClient('mongodb://id:pw@ip.ip.ip.ip:port/')
db = client.youtube 
collection = db.crawal

Writing you_crawal/you_crawal/mongodb.py


In [87]:
%%writefile you_crawal/you_crawal/pipelines.py
from .mongodb import collection

class YoutubeDBPipeline(object):

    def process_item(self, item, spider):

        # mongodb에 저장
        data = {"title": item["title"], 
                "category": item["user_ids"], 
                "content": item["comments"], 
                "link": item["like_nums"]}
        collection.insert(data)

        return item

Overwriting you_crawal/you_crawal/pipelines.py


In [88]:
!tail -n 3 you_crawal/you_crawal/settings.py

#HTTPCACHE_DIR = 'httpcache'
#HTTPCACHE_IGNORE_HTTP_CODES = []
#HTTPCACHE_STORAGE = 'scrapy.extensions.httpcache.FilesystemCacheStorage'


In [89]:
!echo "ITEM_PIPELINES = {" >> you_crawal/you_crawal/settings.py

In [93]:
!echo "   'you_crawal.pipelines.YoutubeDBPipeline': 300," >> you_crawal/you_crawal/settings.py

In [94]:
!echo "}" >> you_crawal/you_crawal/settings.py

In [95]:
!tail -n 3 you_crawal/you_crawal/settings.py

}
   'you_crawal.pipelines.YoutubeDBPipeline': 300,
}
